
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [2]:
batch_size = 10
epochs = 10

In [60]:
linear_layer_2d = nn.Linear(in_features=68, out_features=128)
flatten = nn.Flatten()
input_2d = torch.randn(2, 2, 68) # n, channel, features
#print(input_2d)
print(input_2d.shape)
output_2d = linear_layer_2d(input_2d)
print('output_2d' , output_2d)
#print(output_2d.size())

#output_1d = flatten(input_2d)
#print(output_1d.size())
#linear_layer_2d(output_1d)
#print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)



torch.Size([2, 2, 68])
output_2d tensor([[[-6.3727e-01, -6.4303e-01, -7.5010e-01,  3.6031e-01, -9.4603e-01,
           6.5386e-01, -5.4889e-01, -2.2606e-01, -2.3648e-01, -1.0633e+00,
          -1.3182e-01, -7.2447e-01, -3.8411e-01, -6.1179e-01, -3.0288e-01,
          -1.4347e-01, -3.2634e-01,  6.7208e-02, -5.1199e-01, -1.3262e-01,
          -3.0489e-01, -6.0575e-01, -3.9401e-01, -1.7706e-01, -5.2545e-01,
          -9.4922e-02,  4.3826e-02, -1.5253e-01,  3.0105e-01,  2.1141e-01,
          -2.3795e-02,  4.3036e-01, -1.3217e-01,  1.8488e-02,  9.4067e-03,
           3.0157e-01,  1.1920e-01, -7.0417e-01,  1.6482e-01, -7.7173e-01,
           5.6371e-01,  4.2827e-02, -7.0549e-01,  8.6433e-01,  5.3368e-01,
          -1.0552e-01, -4.1446e-03, -2.1007e-01, -2.3585e-01,  8.8352e-01,
          -2.6643e-01, -1.6444e-01, -8.3990e-01, -1.1832e-01,  1.3146e-01,
           5.0611e-01,  1.0961e+00, -3.9928e-01,  3.8632e-01, -3.0685e-01,
          -8.9605e-02, -3.3475e-04,  1.0583e+00, -4.8248e-01,  2.35

In [4]:

testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)


[[0.15214901 0.26409625]
 [0.15847385 0.35434336]
 [0.17404998 0.4446534 ]
 [0.19193892 0.53497918]
 [0.21678205 0.62303933]
 [0.262472   0.70661546]
 [0.32675891 0.77644054]
 [0.40500139 0.83479593]
 [0.48582416 0.85384914]
 [0.56685146 0.84283572]
 [0.6481305  0.79481721]
 [0.71564276 0.73051458]
 [0.7716696  0.65456918]
 [0.80464692 0.56690236]
 [0.82840444 0.47454697]
 [0.8521777  0.37987876]
 [0.86440259 0.28281907]
 [0.20106432 0.21354466]
 [0.24754095 0.18147987]
 [0.30772143 0.17495049]
 [0.36313466 0.189205  ]
 [0.41614068 0.21732068]
 [0.58269505 0.21382786]
 [0.64068857 0.18878019]
 [0.70321333 0.17764091]
 [0.76562795 0.18269135]
 [0.81392966 0.22233966]
 [0.50349283 0.29655437]
 [0.50303656 0.3636261 ]
 [0.50025174 0.43299491]
 [0.4951541  0.50234799]
 [0.43711337 0.5343341 ]
 [0.46480426 0.54377418]
 [0.49713651 0.5509329 ]
 [0.52726608 0.54188616]
 [0.55509857 0.53051087]
 [0.27217955 0.29960666]
 [0.30701342 0.2790273 ]
 [0.35326978 0.27934197]
 [0.39471171 0.30737899]


In [3]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")   
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


Shape of X [N, F, C]: torch.Size([10, 68, 2])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
X type torch.float32
y type torch.float32


In [4]:
#
#dataiter = iter(train_loader)
#landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(68, 32)
#x = flatten(landmark)
#print(x)
#print(x.size())
# print(landmark.shape)
# print('linear1', x.shape)


In [4]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
            nn.ReLU(),
        )
    

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=33, bias=True)
    (3): ReLU()
  )
)


In [5]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        print(X.shape)
        print('x Type', X.dtype)
        print('y Type', y.dtype)
        # Compute prediction error
        pred = model(X)
        print(pred)       
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
# Training
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
torch.Size([10, 68, 2])
x Type torch.float32
y Type torch.float32
tensor([[7.4872e-02, 1.6131e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6069e-01,
         0.0000e+00, 0.0000e+00, 9.8121e-02, 2.2264e-01, 2.2159e-01, 0.0000e+00,
         0.0000e+00, 8.2138e-02, 3.0891e-02, 3.3933e-01, 0.0000e+00, 2.6585e-01,
         2.1357e-01, 9.6891e-02, 0.0000e+00, 0.0000e+00, 9.4043e-02, 1.3264e-01,
         0.0000e+00, 9.0470e-02, 4.0776e-01, 0.0000e+00, 1.6826e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 2.1752e-02],
        [7.2103e-02, 1.6643e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6023e-01,
         0.0000e+00, 0.0000e+00, 9.5390e-02, 2.2029e-01, 2.1931e-01, 0.0000e+00,
         0.0000e+00, 8.3169e-02, 3.3143e-02, 3.3639e-01, 0.0000e+00, 2.6527e-01,
         2.1064e-01, 9.5317e-02, 0.0000e+00, 0.0000e+00, 9.7943e-02, 1.2894e-01,
         0.0000e+00, 8.5523e-02, 4.0377e-01, 0.0000e+00, 1.6941e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 2.082

In [8]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            print(pred)



In [10]:
for t in range(epochs):
    test(test_loader, model, criterion)

print("Done !")

tensor([[0.2543, 0.2752, 0.0000, 0.0000, 0.0000, 0.3489, 0.0000, 0.0000, 0.2932,
         0.4641, 0.5000, 0.1912, 0.0000, 0.3288, 0.4311, 0.4625, 0.0000, 0.4258,
         0.4713, 0.3251, 0.0000, 0.0000, 0.3680, 0.4595, 0.3249, 0.2964, 0.5065,
         0.0000, 0.4449, 0.0000, 0.0000, 0.0000, 0.3361],
        [0.2508, 0.2777, 0.0000, 0.0000, 0.0000, 0.3457, 0.0000, 0.0000, 0.2933,
         0.4624, 0.4925, 0.1885, 0.0000, 0.3267, 0.4287, 0.4624, 0.0000, 0.4269,
         0.4668, 0.3150, 0.0000, 0.0000, 0.3667, 0.4504, 0.3195, 0.2891, 0.5008,
         0.0000, 0.4427, 0.0000, 0.0000, 0.0000, 0.3333],
        [0.2508, 0.2770, 0.0000, 0.0000, 0.0000, 0.3469, 0.0000, 0.0000, 0.2918,
         0.4636, 0.4930, 0.1856, 0.0000, 0.3229, 0.4293, 0.4666, 0.0000, 0.4248,
         0.4663, 0.3194, 0.0000, 0.0000, 0.3674, 0.4517, 0.3210, 0.2912, 0.5044,
         0.0000, 0.4435, 0.0000, 0.0000, 0.0000, 0.3363],
        [0.2507, 0.2778, 0.0000, 0.0000, 0.0000, 0.3469, 0.0000, 0.0000, 0.2924,
         0.4593,